In [24]:
import os
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ''
os.environ["OPENAI_API_BASE"] = 'https://api.moonshot.cn/v1/'

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")

print("api_key=" + api_key, "base_url=" + base_url)

# 初始化模型(两种方式)：

llm = OpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

chat = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

api_key=sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG base_url=https://api.moonshot.cn/v1/


In [25]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [26]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [27]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [28]:
llm_with_tools = chat.bind_tools([multiply])

In [29]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg

AIMessage(content="To find out what 5 times forty two is, I will use the `multiply` function. I'll provide it with `5` as the first integer and `42` as the second integer.", additional_kwargs={'tool_calls': [{'id': 'multiply:0', 'function': {'arguments': '{\n  "first_int": 5,\n  "second_int": 42\n}', 'name': 'multiply'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 73, 'total_tokens': 138}, 'model_name': 'moonshot-v1-8k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None})

In [30]:
msg

AIMessage(content="To find out what 5 times forty two is, I will use the `multiply` function. I'll provide it with `5` as the first integer and `42` as the second integer.", additional_kwargs={'tool_calls': [{'id': 'multiply:0', 'function': {'arguments': '{\n  "first_int": 5,\n  "second_int": 42\n}', 'name': 'multiply'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 73, 'total_tokens': 138}, 'model_name': 'moonshot-v1-8k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None})

In [31]:
msg.additional_kwargs

{'tool_calls': [{'id': 'multiply:0',
   'function': {'arguments': '{\n  "first_int": 5,\n  "second_int": 42\n}',
    'name': 'multiply'},
   'type': 'function',
   'index': 0}]}

In [32]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

AttributeError: 'AIMessage' object has no attribute 'tool_calls'

In [42]:
from langchain_core.tools import tool


# 从函数创建自定义工具
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base ** exponent


tools = [multiply, add, exponentiate]

In [36]:
from langchain import hub

In [37]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [38]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

ImportError: cannot import name 'create_tool_calling_agent' from 'langchain.agents' (/Users/pangmengting/venv/lib/python3.10/site-packages/langchain/agents/__init__.py)

In [54]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = chat.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

print(tools)

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

[StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Multiply two integers together.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x12f1840d0>), StructuredTool(name='exponentiate', description='exponentiate(base: int, exponent: int) -> int - Exponentiate the base to the exponent power.', args_schema=<class 'pydantic.v1.main.exponentiateSchema'>, func=<function exponentiate at 0x12c0223b0>), StructuredTool(name='add', description='add(first_int: int, second_int: int) -> int - Add two integers.', args_schema=<class 'pydantic.v1.main.addSchema'>, func=<function add at 0x12f184040>)]


In [51]:
chain.invoke("What's 23 times 7")

TypeError: call_tools() missing 1 required positional argument: 'tool'

In [47]:
# 0.1.33
# 0.1.46
!pip show langchain_core  

Name: langchain-core
Version: 0.1.46
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/pangmengting/venv/lib/python3.10/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity
Required-by: langchain, langchain-community, langchain-experimental, langchain-openai, langchain-text-splitters


In [55]:
chain.invoke("add a million plus a billion")

AttributeError: 'AIMessage' object has no attribute 'tool_calls'